In [2]:
import numpy as np
import pandas as pd
import scipy as sc
import re

## Day 16

In [149]:
file16 = open('day16_input', 'r') 
input16_lines = file16.readlines() 

def proc_ticket(ticket_string):

    ticket_list = ticket_string.replace('\n', '').split(',')
    return [int(s) for s in ticket_list]

def proc_field(field):
    
    field_ranges = re.findall(r'\d+', field)
    field_ranges = [int(s) for s in field_ranges]
    field_name   = field.split(':')[0]
    
    return [field_name] + field_ranges

fields_rules = [proc_field(field) for field in input16_lines[:20]]

my_ticket = np.array(proc_ticket(input16_lines[22]))

nearby_tickets = [proc_ticket(ticket) for ticket in input16_lines[25:]]
nearby_tickets = np.array(nearby_tickets)

### Part A

In [150]:
valid_numbers = []
for name, min1, max1, min2, max2 in fields_rules:
    
    valid_numbers += list(range(min1, max1+1)) + list(range(min2, max2+1))
    
valid_numbers = np.unique(valid_numbers)
print('Valid numbers conists of one range only:', (np.diff(valid_numbers) == 1).all())

Valid numbers conists of one range only: True


In [151]:
min_total = valid_numbers.min()
max_total = valid_numbers.max()
print('All numbers on the tickets have to be within', min_total, 'and', max_total)

All numbers on the tickets have to be within 25 and 973


In [152]:
nearby_tickets_false_fields = (nearby_tickets < min_total) | (nearby_tickets > max_total)
nearby_tickets_invalid = nearby_tickets_false_fields.any(axis=1)

In [153]:
scanning_error_rate = nearby_tickets[nearby_tickets_false_fields].sum()
answer16A = scanning_error_rate
print('Answer16A:', answer16A)

Answer16A: 19087


### Part B

Remove invalid tickets, find out which field coorespond that which name.

In [154]:
def check_two_range(arr, min1, max1, min2, max2):
    
    arr_valid_entries = ((arr >= min1) & (arr <= max1)) | ((arr >= min2) & (arr <= max2))
    
    return arr_valid_entries.all(axis=0)

# Remove invalid tickets, add own ticket.
valid_nearby_tickets = nearby_tickets[~nearby_tickets_invalid]
#valid_nearby_tickets = np.append([my_ticket], valid_nearby_tickets, axis=0)

In [155]:
np.array(fields_rules)

array([['departure location', '47', '874', '885', '960'],
       ['departure station', '25', '616', '622', '964'],
       ['departure platform', '42', '807', '825', '966'],
       ['departure track', '36', '560', '583', '965'],
       ['departure date', '37', '264', '289', '968'],
       ['departure time', '27', '325', '346', '954'],
       ['arrival location', '37', '384', '391', '950'],
       ['arrival station', '35', '233', '244', '963'],
       ['arrival platform', '26', '652', '675', '949'],
       ['arrival track', '41', '689', '710', '954'],
       ['class', '27', '75', '81', '952'],
       ['duration', '45', '784', '807', '967'],
       ['price', '40', '350', '374', '970'],
       ['route', '30', '892', '904', '968'],
       ['row', '47', '144', '151', '957'],
       ['seat', '28', '750', '773', '973'],
       ['train', '30', '456', '475', '950'],
       ['type', '34', '642', '648', '968'],
       ['wagon', '42', '486', '498', '970'],
       ['zone', '37', '152', '167', '973']

In [156]:
field_valid_for_all_tickets = []
for field_name, min1, max1, min2, max2 in fields_rules:
    field_valid_for_all_tickets += [list(check_two_range(valid_nearby_tickets, min1, max1, min2, max2))]
    
field_valid_for_all_tickets = np.array(field_valid_for_all_tickets)
valid_fields = pd.DataFrame(field_valid_for_all_tickets.T, columns = np.array(fields_rules).T[0], index=range(20))

In [172]:
a = valid_fields.copy()
class_field_idx_pair = []

for idx in range(20):
    #print(idx)
    ticket_field_single_valid = a.sum() == 1
    ticket_field_single_valid_name = ticket_field_single_valid[ticket_field_single_valid].index[0]
    
    #print(ticket_field_single_valid)
    ticket_field_valid = a.loc[:,ticket_field_single_valid]
    #print(ticket_field_valid)
    ticket_field_valid_idx = np.where(ticket_field_valid)[0]
    #print('\n', ticket_field_valid_idx)
    a.iloc[ticket_field_valid_idx,:] = ticket_field_single_valid.values
    a = a.loc[:,~ticket_field_single_valid.values]
    
    class_field_idx_pair += [[ticket_field_single_valid_name, ticket_field_valid_idx[0]]]
    
class_field_idx_pair = np.array(class_field_idx_pair)

In [180]:
for ticket_class, idx in class_field_idx_pair:
    print(ticket_class, ' - ', my_ticket[int(idx)])

zone  -  61
duration  -  73
type  -  167
arrival station  -  113
class  -  101
price  -  131
arrival location  -  107
departure date  -  97
departure location  -  139
departure track  -  137
departure station  -  71
departure platform  -  127
departure time  -  83
row  -  109
arrival track  -  67
train  -  103
seat  -  53
route  -  89
arrival platform  -  151
wagon  -  59


In [181]:
answer16B = 97 * 139 * 137 * 71 * 127 * 83

In [182]:
answer16B

1382443095281